In [ ]:
import torch

# Parameters

In [ ]:
BATCH_SIZE = 5
N_EPOCHS = 10
LEARNING_RATE = 1e-3 # this equals to 0.001
WEIGHT_DECAY = 1e-3 # this equals to 0.001


# 1. Prepare Data

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [ ]:
mnist_train = MNIST(root="../mnist_tr/", download=True, transform=ToTensor())


Next, we should define a validation dataset:

In [ ]:
# generate indices: instead of the actual data we pass in integers instead
train_indices, val_indices, _, _ = train_test_split(
    range(len(mnist_train)),
    mnist_train.targets,
    test_size=0.1,
)

# generate subset based on indices
tr_dataset = Subset(mnist_train, train_indices)
val_dataset = Subset(mnist_train, val_indices)

tr_dataloader = DataLoader(tr_dataset, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# 2. Model Design

In [ ]:
from torch.nn import Module, Sequential, Linear, ReLU, Conv2d, MaxPool2d, AdaptiveMaxPool2d
from torch import Tensor # Only for type-annotations

Layers: BatchNorm, ReLU, Linear layers

In [ ]:
class ImageClassifier(Module):
    def __init__(self, n_channels: int = 1) -> None:
        super().__init__()
        self.convs = Sequential(
            Conv2d(n_channels, 6, 5),
            ReLU(),
            MaxPool2d(2, 2),
            Conv2d(6, 16, 5),
            ReLU(),
            AdaptiveMaxPool2d(5),
        )
        self.fcs = Sequential(
            Linear(16 * 5 * 5, 120), 
            ReLU(),
            Linear(120, 84), 
            ReLU(),
            Linear(84, 10),
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.convs(x)
        x = torch.flatten(x, 1)
        x = self.fcs(x)
        return x

# 3. Training: optimizers and backpropagation

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("mps" if torch.cuda.is_available() else "cpu") # If on macos

First, lets pick an optimizer and loss function:

In [ ]:
# Define a model:
model = ImageClassifier().to(DEVICE)

# Define loss:
loss_fn = torch.nn.CrossEntropyLoss()

# Define the optimizer, you need to attach your model(s) parameters
# to an optimizer object that will be responsible for updating your model.
optimizer = torch.optim.AdamW(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)

Note that PyTorch has 2 different Adam optimizers, Adam is ... and AdamW is ...

Now we can run the training loop:

In [ ]:
for epoch in range(N_EPOCHS):
    tr_losses = []
    for input_data, target_data in tr_dataloader:
        # Dont forget to move your tensors to your device of preferrence !
        input_data = input_data.to(DEVICE)
        target_data = target_data.to(DEVICE)
        pred_data = model(input_data)
        tr_loss = loss_fn(pred_data, target_data)

        # First step is to clear the calculated gradients from the previous iteration.
        optimizer.zero_grad()

        # Loss should be 1-value tensor and is a leaf in the gradient tree,
        # Based on the gradient functions attached to the tensors (grad_fn), automatic
        # backpropagation will calculate all gradients in the background.
        tr_loss.backward()

        # Optimizer object is responsible for updating model weights and biases,
        # after the gradients are calculated.
        optimizer.step()

        # tensor.detach() function breaks the gradient tree and returns the tensor only including the data !
        # Thats why you are only allowed to do that after you finish the gradient calculations.
        tr_losses.append(tr_loss.detach())
    
    # tensor.item() returns the one and only attached value here: e.g., tensor((0.00135)) -> 0.00135
    avg_tr_loss = torch.stack(tr_losses).mean().item()

    # After a successful training iteration, we can run a validation loop to monitor model's performance:
    # Since there will be no backpropagation on the validation step, it is unnecessary to store gradients.
    # Therefore we use torch.no_grad to disable all gradient functions during validation for memory efficiency.
    with torch.no_grad():
        val_losses = []
        for input_data, target_data in val_dataloader:
            # Dont forget to move your tensors to your device of preferrence !
            input_data = input_data.to(DEVICE)
            target_data = target_data.to(DEVICE)
            pred_data = model(input_data)
            val_loss = loss_fn(pred_data, target_data)
            val_losses.append(val_loss.detach())
        avg_val_loss = torch.stack(val_losses).mean().item()

    print(f"Epoch: {epoch + 1}/{N_EPOCHS}, Avg.Tr.Loss: {avg_tr_loss}, Avg.Val.Loss: {avg_val_loss}")

# 4. Testing and evaluation

It is crucial that your testing set is completely disjoint from your training and validation sets, plus it should have the correct representation of the use cases of your model in real life.

In [ ]:
# Collect test results one by one (no-batch), because they will be used to calculate metrics.
mnist_test = MNIST(root="../mnist_test/", download=True, train=False, transform=ToTensor())

test_dataloader = DataLoader(mnist_test, batch_size=1)

Testing loop:

In [ ]:
with torch.no_grad():
    test_preds = []
    test_logits = []
    test_labels = []
    for input_data, target_data in test_dataloader:
        # Dont forget to move your tensors to your device of preferrence !
        input_data = input_data.to(DEVICE)
        target_data = target_data.to(DEVICE)
        pred_data = model(input_data).squeeze()
        test_logits.append(torch.softmax(pred_data, 0))
        pred_class = torch.argmax(pred_data)
        test_preds.append(pred_class)
        test_labels.append(target_data)

    logit_preds = torch.stack(test_logits).cpu().numpy()
    class_preds = torch.stack(test_preds).cpu().numpy()
    class_labels = torch.stack(test_labels).cpu().numpy()


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay

Here lets take a look at the basic classification metrics: accuracy and f1-score:

In [ ]:
acc = accuracy_score(class_labels, class_preds)
f1 = f1_score(class_labels, class_preds, average="weighted")
roc_auc = roc_auc_score(class_labels, logit_preds, multi_class="ovr")
acc, f1, roc_auc

We need to visualize our results for better interpretation:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Then we can plot a ROC-AUC curve to investigate the model confidence on its predictions.

In [ ]:
# roc_curve(class_labels[np.where(class_labels==0)], logit_preds[np.where(class_labels==0)])


Lastly, the multi-class confusion matrix is crucial to see the model behaviour.

In [ ]:
conf_mat = confusion_matrix(class_labels, class_preds)
print(conf_mat)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
disp.plot()
plt.show()

# 5. Model inference

Now use your model in production ! Dont forget to apply the preprocessing steps if there is any.